In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
#IMPORT WAITING LIST
url = r'C:\Users\achie\Github\NHS-Surgery\V2'

url = url + r'\Input.xlsx'
Patient_list = pd.read_excel(url, sheet_name = 0, header=0, index_col=0)
Theatres = pd.read_excel(url, sheet_name = 1, header=0, index_col=0)

%store -r NDays

start = dt.datetime(2020,12,28)
end = start + dt.timedelta(days=NDays)

In [3]:
#GET NEW WAITING TIME

today = dt.datetime.today()

Patient_list['Waiting_time'] = (today - Patient_list['Date of Admission']).dt.days

Patient_list['Welfare_cost'] = Patient_list['Waiting_time'] * Patient_list['Hospital Urgency Score']

Patient_list['New_op_time'] = np.where(Patient_list['Specialty Descriptions'] == 'Ophthalmology',
                                       Patient_list['Expected length of operation (mins)'] + 5,
                                       Patient_list['Expected length of operation (mins)'] + 15)

Patient_list['Patient_ID'] = Patient_list.index

In [4]:
#BY CLINICAL URGENCY EFFICIENCY

Patient_list['Clinical_efficiency'] = (Patient_list['Hospital Urgency Score'] / 
                                       Patient_list['New_op_time'])

In [5]:
#BY WAITING EFFICIENCY

Patient_list['Waiting_efficiency'] = (Patient_list['Waiting_time'] / 
                                       Patient_list['New_op_time'])

In [6]:
#BY WELFARE EFFICIENCY

Patient_list['Welfare_efficiency'] = (Patient_list['Welfare_cost'] / 
                                       Patient_list['New_op_time'])

In [7]:
#ORDER

Clinical_urgency = Patient_list.sort_values(by='Hospital Urgency Score', ascending=False)
Clinical_efficiency = Patient_list.sort_values(by='Clinical_efficiency', ascending=False)
Waiting_urgency = Patient_list.sort_values(by='Waiting_time', ascending=False)
Waiting_efficiency = Patient_list.sort_values(by='Waiting_efficiency', ascending=False)
Welfare_urgency = Patient_list.sort_values(by='Welfare_cost', ascending=False)
Welfare_efficiency = Patient_list.sort_values(by='Welfare_efficiency', ascending=False)
Number = Patient_list.sort_values(by='New_op_time', ascending=True)

In [8]:
#SAVE

url = r'C:\Users\achie\Github\NHS-Surgery\V2\Patient List - Sorted.xlsx'
writer = pd.ExcelWriter(url , engine='xlsxwriter')
Clinical_urgency.to_excel(writer, sheet_name='Clinical Urgency')
Clinical_efficiency.to_excel(writer, sheet_name='Clinical Efficiency')
Waiting_urgency.to_excel(writer, sheet_name='Waiting Urgency')
Waiting_efficiency.to_excel(writer, sheet_name='Waiting Efficiency')
Welfare_urgency.to_excel(writer, sheet_name='Welfare Urgency')
Welfare_efficiency.to_excel(writer, sheet_name='Welfare Efficiency')
Number.to_excel(writer, sheet_name='Volume')
writer.save()